<h2 align='center'>Codebasics ML Course: ML Flow Tutorial</h2>


In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings

warnings.filterwarnings("ignore")


In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(
    n_samples=1000,
    n_features=10,
    n_informative=2,
    n_redundant=8,
    weights=[0.9, 0.1],
    flip_y=0,
    random_state=42,
)

np.unique(y, return_counts=True)


(array([0, 1]), array([900, 100], dtype=int64))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)


#### Handle class imbalance


In [4]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(X_train_res, return_counts=True)


(array([-5.51207083, -5.14120195, -4.69421946, ...,  5.84871724,
         5.91086505,  6.02415826]),
 array([1, 1, 1, ..., 1, 1, 1], dtype=int64))

### Track Experiments


In [5]:
models = [
    (
        "Logistic Regression",
        {"C": 1, "solver": "liblinear"},
        LogisticRegression(),
        (X_train, y_train),
        (X_test, y_test),
    ),
    (
        "Random Forest",
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(),
        (X_train, y_train),
        (X_test, y_test),
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": "logloss"},
        XGBClassifier(),
        (X_train, y_train),
        (X_test, y_test),
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": "logloss"},
        XGBClassifier(),
        (X_train_res, y_train_res),
        (X_test, y_test),
    ),
]


In [6]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]

    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)


In [7]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost


## Dags hub setup


In [10]:
# # Ideally you will not require following 4 lines if you have started fresh and do not have any previous dagshub credentials on your computer
# import os
# os.environ['MLFLOW_TRACKING_USERNAME'] = 'your user name' # 'learnpythonlanguage'
# os.environ['MLFLOW_TRACKING_PASSWORD'] = 'your password' #
# os.environ['MLFLOW_TRACKING_URI'] = 'your dagshub unique uri' # https://dagshub.com/learnpythonlanguage/mlflow_dagshub_demo.mlflow


In [ ]:
# %pip install dagshub
import dagshub
import mlflow

dagshub.init(repo_owner="DhruvJais01", repo_name="mlflow_dagshub_demo", mlflow=True)


with mlflow.start_run():
    mlflow.log_param("parameter name", "value")
    mlflow.log_metric("metric name", 1)


   ---------------------------------------- 0.0/13.4 MB ? eta -:--:--
   ---------- ----------------------------- 3.7/13.4 MB 19.8 MB/s eta 0:00:01
   ------------- -------------------------- 4.5/13.4 MB 12.2 MB/s eta 0:00:01
   ---------------- ----------------------- 5.5/13.4 MB 8.8 MB/s eta 0:00:01
   ------------------- -------------------- 6.6/13.4 MB 7.7 MB/s eta 0:00:01
   ----------------------- ---------------- 7.9/13.4 MB 7.4 MB/s eta 0:00:01
   -------------------------- ------------- 8.9/13.4 MB 7.0 MB/s eta 0:00:01
   ----------------------------- ---------- 10.0/13.4 MB 6.7 MB/s eta 0:00:01
   ------------------------------- -------- 10.7/13.4 MB 6.3 MB/s eta 0:00:01
   ---------------------------------- ----- 11.5/13.4 MB 6.2 MB/s eta 0:00:01
   -------------------------------------- - 12.8/13.4 MB 6.0 MB/s eta 0:00:01
   ---------------------------------------- 13.4/13.4 MB 6.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.3

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=05e73365-4bc7-46c2-a0be-576c98365309&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=a86c339ab013427fa3df1ad980200fee2555a8e4886c5f2595f16763e7d813bb




Accessing as DhruvJais01

Initialized MLflow to track repo "DhruvJais01/mlflow_dagshub_demo"

Repository DhruvJais01/mlflow_dagshub_demo initialized!

🏃 View run welcoming-wolf-263 at: https://dagshub.com/DhruvJais01/mlflow_dagshub_demo.mlflow/#/experiments/0/runs/0ebcdfe24e07499ba6759a60fef1b2f8
🧪 View experiment at: https://dagshub.com/DhruvJais01/mlflow_dagshub_demo.mlflow/#/experiments/0


In [9]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection V1")
# mlflow.set_tracking_uri("http://localhost:5000") not in local
mlflow.set_tracking_uri("https://dagshub.com/DhruvJais01/mlflow_dagshub_demo.mlflow")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]

    with mlflow.start_run(run_name=model_name):
        mlflow.log_params(params)
        mlflow.log_metrics(
            {
                "accuracy": report["accuracy"],
                "recall_class_1": report["1"]["recall"],
                "recall_class_0": report["0"]["recall"],
                "f1_score_macro": report["macro avg"]["f1-score"],
            }
        )

        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")


2025/03/29 15:21:52 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly Detection V1' does not exist. Creating a new experiment.
2025/03/29 15:22:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: https://dagshub.com/DhruvJais01/mlflow_dagshub_demo.mlflow/#/experiments/1/runs/c6f54c3ae9914af1a03080fda1d6db7d
🧪 View experiment at: https://dagshub.com/DhruvJais01/mlflow_dagshub_demo.mlflow/#/experiments/1


2025/03/29 15:22:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: https://dagshub.com/DhruvJais01/mlflow_dagshub_demo.mlflow/#/experiments/1/runs/537d8c2eb5e24f7ba64ad39d82596f82
🧪 View experiment at: https://dagshub.com/DhruvJais01/mlflow_dagshub_demo.mlflow/#/experiments/1


2025/03/29 15:22:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: https://dagshub.com/DhruvJais01/mlflow_dagshub_demo.mlflow/#/experiments/1/runs/5d253fb08a3447fd96a36771e62b4cdd
🧪 View experiment at: https://dagshub.com/DhruvJais01/mlflow_dagshub_demo.mlflow/#/experiments/1


2025/03/29 15:22:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: https://dagshub.com/DhruvJais01/mlflow_dagshub_demo.mlflow/#/experiments/1/runs/1957968b287e4712a0705dd65514257d
🧪 View experiment at: https://dagshub.com/DhruvJais01/mlflow_dagshub_demo.mlflow/#/experiments/1


### Register the Model


In [11]:
model_name = "XGB-Smote"
run_id = "1957968b287e4712a0705dd65514257d"
model_uri = f"runs:/{run_id}/model"

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)


Successfully registered model 'XGB-Smote'.
2025/03/29 15:26:00 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1
Created version '1' of model 'XGB-Smote'.


🏃 View run XGBClassifier With SMOTE at: https://dagshub.com/DhruvJais01/mlflow_dagshub_demo.mlflow/#/experiments/1/runs/1957968b287e4712a0705dd65514257d
🧪 View experiment at: https://dagshub.com/DhruvJais01/mlflow_dagshub_demo.mlflow/#/experiments/1


### Load the Model


In [12]:
# load directly
import mlflow
import mlflow.xgboost
import pandas as pd

# model_uri = "runs:/3ad3473128104944b04619c6aa2ee69e/model"

# Load model explicitly
loaded_model = mlflow.xgboost.load_model(model_uri)

# Ensure input_data is correctly formatted
input_data = pd.DataFrame(X_test)  # Convert to DataFrame

# Make predictions
y_pred = loaded_model.predict(input_data)
print("First 4 Predictions:", y_pred[:4])


First 4 Predictions: [0 0 0 0]


In [13]:
# load with version
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]


array([0, 0, 0, 0])

In [14]:
# load with alias
model_uri = f"models:/{model_name}@challanger"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]


array([0, 0, 0, 0])

### Transition the Model to Production


In [15]:
current_model_uri = f"models:/{model_name}@challanger"
production_model_name = "anomaly-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(
    src_model_uri=current_model_uri, dst_name=production_model_name
)


Successfully registered model 'anomaly-detection-prod'.
Copied version '1' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1743242228609, current_stage='None', description='', last_updated_timestamp=1743242228609, name='anomaly-detection-prod', run_id='1957968b287e4712a0705dd65514257d', run_link='', source='models:/XGB-Smote/1', status='READY', status_message=None, tags={}, user_id='', version='1'>

## Loading Production Model


In [16]:
model_version = 1
prod_model_uri = f"models:/{production_model_name}@champion"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]


array([0, 0, 0, 0])

Please refer to following to learn more about model registry

https://mlflow.org/docs/latest/model-registry.html#model-registry-workflows to learn
